# 🧭 QAOA para TSP no IBM Quantum

## Execução em Hardware Quântico Real

Este notebook implementa o TSP usando QAOA para execução no **IBM Quantum Cloud**.

### Limitações de Hardware
- **3 cidades**: 9 qubits ✅
- **4 cidades**: 16 qubits ✅

### Diferenças do Simulador
- Execução em hardware quântico real (com ruído)
- Uso do Qiskit Runtime (Sampler)
- Tempo de fila para execução
- Resultados podem variar devido ao ruído

---
## 📦 Instalação e Imports

In [ ]:
# Instalar/atualizar pacotes necessários
# Descomente se necessário

# !pip install qiskit>=1.0
# !pip install qiskit-ibm-runtime
# !pip install qiskit-aer

In [1]:
# Imports
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Session
from qiskit_aer import AerSimulator
import numpy as np
from scipy.optimize import minimize
from itertools import permutations
import matplotlib.pyplot as plt
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


---
## 🔑 Configuração do IBM Quantum

### Como obter seu token:
1. Acesse [quantum.ibm.com](https://quantum.ibm.com/)
2. Faça login ou crie uma conta
3. Vá em "Account settings" → "API token"
4. Copie o token e cole abaixo

In [4]:
# =============================================================================
# CONFIGURAÇÃO IBM QUANTUM
# =============================================================================

# Opção 1: Salvar credenciais (execute apenas uma vez)
QiskitRuntimeService.save_account(
     channel="ibm_quantum",
     token="Qqqq",
     overwrite=True )

# Opção 2: Usar credenciais já salvas
try:
    service = QiskitRuntimeService(channel="ibm_quantum")
    print("✅ Conectado ao IBM Quantum!")
    
    # Listar backends disponíveis
    backends = service.backends()
    print(f"\n📡 Backends disponíveis: {len(backends)}")
    for b in backends[:5]:
        print(f"   - {b.name}: {b.num_qubits} qubits")
    if len(backends) > 5:
        print(f"   ... e mais {len(backends)-5} backends")
        
except Exception as e:
    print(f"❌ Erro ao conectar: {e}")
    print("\n💡 Execute a célula com save_account() primeiro!")

InvalidAccountError: "Invalid `channel` value. Expected one of ['ibm_cloud', 'ibm_quantum_platform'], got 'ibm_quantum'."

In [ ]:
# =============================================================================
# SELECIONAR BACKEND
# =============================================================================

# Opção 1: Usar o backend com menor fila (recomendado)
backend = service.least_busy(min_num_qubits=16, operational=True)
print(f"\n🎯 Backend selecionado: {backend.name}")
print(f"   Qubits: {backend.num_qubits}")

# Opção 2: Escolher backend específico (descomente para usar)
# backend = service.backend("ibm_brisbane")  # 127 qubits
# backend = service.backend("ibm_osaka")     # 127 qubits

# Opção 3: Usar simulador do IBM (sem fila, para testes)
# backend = service.backend("ibmq_qasm_simulator")

---
## 📊 Definição do Problema TSP

Matrizes de distância para 3 e 4 cidades (compatíveis com hardware quântico atual).

In [ ]:
# Matrizes de distâncias (apenas 3 e 4 cidades para hardware real)
graphs = {
    3: np.array([
        [0, 10, 15],
        [10, 0, 20],
        [15, 20, 0]
    ], dtype=float),
    
    4: np.array([
        [0, 1, 50, 50],
        [1, 0, 2, 50],
        [50, 2, 0, 3],
        [50, 50, 3, 0]
    ], dtype=float)
}

for n, D in graphs.items():
    print(f"\n📍 Matriz D ({n} cidades) - {n**2} qubits:")
    print(D.astype(int))

---
## 🔧 Funções do QAOA para TSP

As mesmas funções do notebook didático, otimizadas para IBM Quantum.

In [ ]:
def qubit_index(cidade, tempo, n):
    """Mapeia (cidade, tempo) para índice do qubit."""
    return cidade * n + tempo


def construir_hamiltoniano_tsp(D, penalty_multiplier=2.0):
    """
    Constrói os coeficientes do Hamiltoniano de custo para o TSP.
    
    H_QUBO = H_dist + A*H_p1 + A*H_p2
    
    Mapeamento: x_{i,t} → (I - Z_{i,t})/2
    """
    n = len(D)
    A = penalty_multiplier * np.max(D) * n
    
    h = {q: 0.0 for q in range(n * n)}  # Termos Z_i
    J = {}  # Termos Z_i Z_j
    
    # H_dist: custo das distâncias
    for i in range(n):
        for j in range(n):
            if i != j:
                d_ij = D[i, j]
                for t in range(n):
                    t_next = (t + 1) % n
                    q_a = qubit_index(i, t, n)
                    q_b = qubit_index(j, t_next, n)
                    
                    h[q_a] -= d_ij / 4
                    h[q_b] -= d_ij / 4
                    key = (min(q_a, q_b), max(q_a, q_b))
                    J[key] = J.get(key, 0) + d_ij / 4
    
    # H_p1: cada cidade visitada uma vez
    for i in range(n):
        for t in range(n):
            q = qubit_index(i, t, n)
            h[q] += A / 2
            for t2 in range(t + 1, n):
                q2 = qubit_index(i, t2, n)
                h[q] -= A / 2
                h[q2] -= A / 2
                key = (min(q, q2), max(q, q2))
                J[key] = J.get(key, 0) + A / 2
    
    # H_p2: cada tempo tem uma cidade
    for t in range(n):
        for i in range(n):
            q = qubit_index(i, t, n)
            h[q] += A / 2
            for i2 in range(i + 1, n):
                q2 = qubit_index(i2, t, n)
                h[q] -= A / 2
                h[q2] -= A / 2
                key = (min(q, q2), max(q, q2))
                J[key] = J.get(key, 0) + A / 2
    
    return h, J, A


def qaoa_layer(qc, h, J, gamma, beta):
    """
    Implementa uma camada do QAOA.
    
    U_C(γ) = exp(-iγ Ĥ_C)
    U_M(β) = exp(-iβ Σ X_k)
    """
    # Operador de Custo U_C(γ)
    for qubit, coef in h.items():
        if abs(coef) > 1e-10:
            qc.rz(2 * gamma * coef, qubit)
    
    for (q_i, q_j), coef in J.items():
        if abs(coef) > 1e-10:
            qc.cx(q_i, q_j)
            qc.rz(2 * gamma * coef, q_j)
            qc.cx(q_i, q_j)
    
    # Operador Mixer U_M(β)
    for q in range(qc.num_qubits):
        qc.rx(2 * beta, q)


def qaoa_circuit(h, J, num_qubits, gammas, betas):
    """
    Constrói o circuito QAOA completo.
    
    |ψ(γ,β)⟩ = Π_{l=1}^{p} U_M(β_l) U_C(γ_l) |+⟩^{⊗n²}
    """
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))  # Estado inicial |+⟩
    
    for gamma, beta in zip(gammas, betas):
        qaoa_layer(qc, h, J, gamma, beta)
    
    return qc


print("✅ Funções QAOA definidas!")

In [ ]:
def decodificar_bitstring(bitstring, n):
    """Decodifica bitstring em rota TSP, verificando restrições."""
    x = np.zeros((n, n), dtype=int)
    for idx, bit in enumerate(bitstring):
        cidade = idx // n
        tempo = idx % n
        x[cidade, tempo] = int(bit)
    
    # Verifica restrições
    for i in range(n):
        if np.sum(x[i, :]) != 1:
            return None, False
    for t in range(n):
        if np.sum(x[:, t]) != 1:
            return None, False
    
    # Extrai rota
    rota = []
    for t in range(n):
        for i in range(n):
            if x[i, t] == 1:
                rota.append(i)
                break
    rota.append(rota[0])
    return tuple(rota), True


def tsp_cost(bitstring, D):
    """Calcula custo de uma solução TSP."""
    n = len(D)
    rota, valida = decodificar_bitstring(bitstring, n)
    
    if not valida:
        return float('inf'), None, False
    
    custo = sum(D[rota[t], rota[t + 1]] for t in range(n))
    return custo, rota, True


def processar_counts(counts, D):
    """
    Processa contagens de medição e extrai resultados.
    """
    n = len(D)
    total_shots = sum(counts.values())
    penalidade = np.max(D) * n * 10
    
    exp_cost = 0
    n_validas = 0
    melhor_custo = float('inf')
    melhor_rota = None
    
    for bitstring, count in counts.items():
        # Remove espaços (formato IBM) e inverte
        bs = bitstring.replace(" ", "")[::-1]
        
        custo, rota, valida = tsp_cost(bs, D)
        
        if valida:
            n_validas += count
            exp_cost += count * custo
            if custo < melhor_custo:
                melhor_custo = custo
                melhor_rota = rota
        else:
            exp_cost += count * penalidade
    
    return {
        'exp_cost': exp_cost / total_shots,
        'frac_validas': n_validas / total_shots,
        'melhor_rota': melhor_rota,
        'melhor_custo': melhor_custo
    }


def brute_force_tsp(D):
    """Resolve TSP por força bruta (solução ótima)."""
    n = len(D)
    melhor_rota = None
    melhor_custo = float('inf')
    
    for perm in permutations(range(1, n)):
        rota = (0,) + perm + (0,)
        custo = sum(D[rota[i], rota[i+1]] for i in range(n))
        if custo < melhor_custo:
            melhor_custo = custo
            melhor_rota = rota
    
    return melhor_rota, melhor_custo


print("✅ Funções auxiliares definidas!")

---
## 🖥️ Execução Local (Simulador) - Para Otimização dos Parâmetros

Primeiro otimizamos γ e β usando simulador local (mais rápido), depois executamos no IBM Quantum com os parâmetros otimizados.

In [ ]:
# Simulador local para otimização
sim_local = AerSimulator()

def objective_local(params, h, J, D, num_qubits, p, shots=2048):
    """
    Função objetivo usando simulador local (para otimização rápida).
    """
    gammas = params[:p]
    betas = params[p:]
    
    qc = qaoa_circuit(h, J, num_qubits, gammas, betas)
    qc.measure_all()
    
    tqc = transpile(qc, sim_local)
    result = sim_local.run(tqc, shots=shots).result()
    counts = result.get_counts()
    
    res = processar_counts(counts, D)
    return res['exp_cost']


def otimizar_parametros(D, p=2, maxiter=200):
    """
    Otimiza parâmetros γ e β usando simulador local.
    """
    n = len(D)
    num_qubits = n ** 2
    h, J, A = construir_hamiltoniano_tsp(D)
    
    print(f"\n🔄 Otimizando parâmetros (simulador local)...")
    print(f"   Cidades: {n}, Qubits: {num_qubits}, Camadas: {p}")
    
    init_params = np.random.uniform(0, np.pi, 2 * p)
    
    resultado = minimize(
        objective_local,
        init_params,
        args=(h, J, D, num_qubits, p),
        method="COBYLA",
        options={'maxiter': maxiter}
    )
    
    gammas = resultado.x[:p]
    betas = resultado.x[p:]
    
    print(f"   ✅ Otimização concluída!")
    print(f"   γ = {[f'{g:.4f}' for g in gammas]}")
    print(f"   β = {[f'{b:.4f}' for b in betas]}")
    
    return gammas, betas, h, J


print("✅ Funções de otimização local definidas!")

In [ ]:
# =============================================================================
# OTIMIZAR PARÂMETROS PARA CADA PROBLEMA
# =============================================================================

parametros_otimizados = {}

for n_cidades, D in graphs.items():
    print(f"\n{'='*60}")
    print(f"📍 OTIMIZANDO: {n_cidades} CIDADES")
    print(f"{'='*60}")
    
    gammas, betas, h, J = otimizar_parametros(D, p=2, maxiter=200)
    
    parametros_otimizados[n_cidades] = {
        'gammas': gammas,
        'betas': betas,
        'h': h,
        'J': J,
        'D': D
    }

print(f"\n{'='*60}")
print("✅ Parâmetros otimizados para todas as instâncias!")
print(f"{'='*60}")

---
## ☁️ Execução no IBM Quantum

Agora executamos os circuitos com parâmetros otimizados no hardware quântico real.

In [ ]:
def executar_ibm_quantum(backend, parametros, shots=4096):
    """
    Executa circuitos QAOA no IBM Quantum.
    
    Usa Qiskit Runtime Session para execução eficiente.
    """
    resultados = []
    
    print(f"\n{'='*70}")
    print(f"☁️  EXECUÇÃO NO IBM QUANTUM: {backend.name}")
    print(f"{'='*70}")
    
    # Abrir sessão com o backend
    with Session(service=service, backend=backend) as session:
        sampler = Sampler(session=session)
        
        for n_cidades, params in parametros.items():
            print(f"\n📍 Processando {n_cidades} cidades ({n_cidades**2} qubits)...")
            
            D = params['D']
            h = params['h']
            J = params['J']
            gammas = params['gammas']
            betas = params['betas']
            num_qubits = n_cidades ** 2
            
            # Construir circuito
            qc = qaoa_circuit(h, J, num_qubits, gammas, betas)
            qc.measure_all()
            
            # Transpilar para o backend
            print(f"   🔧 Transpilando circuito...")
            qc_transpiled = transpile(qc, backend=backend, optimization_level=3)
            print(f"   Profundidade após transpilação: {qc_transpiled.depth()}")
            
            # Executar
            print(f"   🚀 Enviando job para {backend.name}...")
            inicio = time.time()
            
            job = sampler.run([qc_transpiled], shots=shots)
            print(f"   Job ID: {job.job_id()}")
            print(f"   ⏳ Aguardando execução...")
            
            # Aguardar resultado
            result = job.result()
            tempo_execucao = time.time() - inicio
            
            # Extrair contagens
            pub_result = result[0]
            counts = pub_result.data.meas.get_counts()
            
            # Processar resultados
            res = processar_counts(counts, D)
            
            # Solução ótima (brute force)
            rota_otima, custo_otimo = brute_force_tsp(D)
            
            # Calcular gap
            if res['melhor_rota'] is not None:
                gap = ((res['melhor_custo'] - custo_otimo) / custo_otimo) * 100
            else:
                gap = float('inf')
            
            print(f"   ✅ Execução concluída em {tempo_execucao:.2f}s")
            print(f"   📊 Resultados:")
            print(f"      Soluções válidas: {100*res['frac_validas']:.2f}%")
            print(f"      Melhor rota QAOA: {res['melhor_rota']}")
            print(f"      Custo QAOA: {res['melhor_custo']}")
            print(f"      Custo ótimo: {custo_otimo}")
            print(f"      Gap: {gap:.2f}%")
            
            resultados.append({
                'Cidades': n_cidades,
                'Qubits': num_qubits,
                'Backend': backend.name,
                'Rota Ótima': str(rota_otima),
                'Custo Ótimo': custo_otimo,
                'Rota QAOA': str(res['melhor_rota']),
                'Custo QAOA': res['melhor_custo'] if res['melhor_custo'] != float('inf') else 'N/A',
                'Gap (%)': gap if gap != float('inf') else 'N/A',
                'Soluções Válidas (%)': res['frac_validas'] * 100,
                'Tempo (s)': tempo_execucao,
                'Shots': shots,
                'Job ID': job.job_id()
            })
    
    return pd.DataFrame(resultados)


print("✅ Função de execução IBM Quantum definida!")

In [ ]:
# =============================================================================
# EXECUTAR NO IBM QUANTUM
# =============================================================================

# ⚠️ ATENÇÃO: Esta célula envia jobs para o IBM Quantum!
# Pode haver fila de espera dependendo do backend.

df_ibm = executar_ibm_quantum(backend, parametros_otimizados, shots=4096)

---
## 📊 Resultados: IBM Quantum

In [ ]:
# Exibir resultados do IBM Quantum
print("\n" + "=" * 70)
print("📊 RESULTADOS - IBM QUANTUM")
print("=" * 70 + "\n")

display(df_ibm)

---
## 🔄 Comparação: Simulador Local vs IBM Quantum

Vamos comparar os resultados do simulador (sem ruído) com o hardware real (com ruído).

In [ ]:
def executar_simulador_local(parametros, shots=4096):
    """
    Executa no simulador local para comparação.
    """
    resultados = []
    
    for n_cidades, params in parametros.items():
        D = params['D']
        h = params['h']
        J = params['J']
        gammas = params['gammas']
        betas = params['betas']
        num_qubits = n_cidades ** 2
        
        # Construir e executar circuito
        qc = qaoa_circuit(h, J, num_qubits, gammas, betas)
        qc.measure_all()
        
        inicio = time.time()
        tqc = transpile(qc, sim_local)
        result = sim_local.run(tqc, shots=shots).result()
        tempo = time.time() - inicio
        
        counts = result.get_counts()
        res = processar_counts(counts, D)
        
        rota_otima, custo_otimo = brute_force_tsp(D)
        
        if res['melhor_rota'] is not None:
            gap = ((res['melhor_custo'] - custo_otimo) / custo_otimo) * 100
        else:
            gap = float('inf')
        
        resultados.append({
            'Cidades': n_cidades,
            'Custo Ótimo': custo_otimo,
            'Custo Simulador': res['melhor_custo'] if res['melhor_custo'] != float('inf') else 'N/A',
            'Gap Simulador (%)': gap if gap != float('inf') else 'N/A',
            'Válidas Simulador (%)': res['frac_validas'] * 100,
            'Tempo Simulador (s)': tempo
        })
    
    return pd.DataFrame(resultados)


# Executar no simulador
df_sim = executar_simulador_local(parametros_otimizados, shots=4096)

print("\n" + "=" * 70)
print("📊 RESULTADOS - SIMULADOR LOCAL (sem ruído)")
print("=" * 70 + "\n")
display(df_sim)

In [ ]:
# =============================================================================
# TABELA COMPARATIVA FINAL
# =============================================================================

print("\n" + "=" * 70)
print("📋 COMPARAÇÃO: SIMULADOR vs IBM QUANTUM vs BRUTE FORCE")
print("=" * 70 + "\n")

# Criar tabela comparativa
df_comparacao = pd.DataFrame()

for n_cidades in graphs.keys():
    D = graphs[n_cidades]
    rota_bf, custo_bf = brute_force_tsp(D)
    
    sim_row = df_sim[df_sim['Cidades'] == n_cidades].iloc[0]
    ibm_row = df_ibm[df_ibm['Cidades'] == n_cidades].iloc[0]
    
    df_comparacao = pd.concat([df_comparacao, pd.DataFrame([{
        'Cidades': n_cidades,
        'Qubits': n_cidades ** 2,
        'Custo Ótimo (BF)': custo_bf,
        'Custo Simulador': sim_row['Custo Simulador'],
        'Custo IBM Quantum': ibm_row['Custo QAOA'],
        'Gap Simulador (%)': sim_row['Gap Simulador (%)'],
        'Gap IBM (%)': ibm_row['Gap (%)'],
        'Válidas Sim (%)': sim_row['Válidas Simulador (%)'],
        'Válidas IBM (%)': ibm_row['Soluções Válidas (%)'],
        'Backend IBM': ibm_row['Backend']
    }])], ignore_index=True)

display(df_comparacao)

In [ ]:
# =============================================================================
# GRÁFICOS COMPARATIVOS
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

x = df_comparacao['Cidades'].values
width = 0.25

# Gráfico 1: Custos
ax1 = axes[0]
custos_sim = [c if c != 'N/A' else 0 for c in df_comparacao['Custo Simulador']]
custos_ibm = [c if c != 'N/A' else 0 for c in df_comparacao['Custo IBM Quantum']]

ax1.bar(x - width, df_comparacao['Custo Ótimo (BF)'], width, label='Brute Force', color='green')
ax1.bar(x, custos_sim, width, label='Simulador', color='steelblue')
ax1.bar(x + width, custos_ibm, width, label='IBM Quantum', color='purple')
ax1.set_xlabel('Cidades')
ax1.set_ylabel('Custo')
ax1.set_title('Comparação de Custos')
ax1.legend()
ax1.set_xticks(x)

# Gráfico 2: Gap
ax2 = axes[1]
gaps_sim = [g if g != 'N/A' else 100 for g in df_comparacao['Gap Simulador (%)']]
gaps_ibm = [g if g != 'N/A' else 100 for g in df_comparacao['Gap IBM (%)']]

ax2.bar(x - width/2, gaps_sim, width, label='Simulador', color='steelblue')
ax2.bar(x + width/2, gaps_ibm, width, label='IBM Quantum', color='purple')
ax2.set_xlabel('Cidades')
ax2.set_ylabel('Gap (%)')
ax2.set_title('Gap Relativo ao Ótimo')
ax2.legend()
ax2.set_xticks(x)
ax2.axhline(y=0, color='green', linestyle='--', alpha=0.5)

# Gráfico 3: Soluções Válidas
ax3 = axes[2]
ax3.bar(x - width/2, df_comparacao['Válidas Sim (%)'], width, label='Simulador', color='steelblue')
ax3.bar(x + width/2, df_comparacao['Válidas IBM (%)'], width, label='IBM Quantum', color='purple')
ax3.set_xlabel('Cidades')
ax3.set_ylabel('Soluções Válidas (%)')
ax3.set_title('Porcentagem de Soluções Válidas')
ax3.legend()
ax3.set_xticks(x)

plt.tight_layout()
plt.savefig('comparacao_simulador_vs_ibm.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📁 Gráfico salvo como 'comparacao_simulador_vs_ibm.png'")

In [ ]:
# =============================================================================
# ANÁLISE FINAL
# =============================================================================

print("\n" + "=" * 70)
print("📈 ANÁLISE: SIMULADOR vs HARDWARE REAL")
print("=" * 70)

print("\n🔹 SIMULADOR (sem ruído):")
print("-" * 50)
print("   • Execução ideal, sem erros de hardware")
print("   • Resultados representam o limite teórico do QAOA")
print(f"   • Média de soluções válidas: {df_comparacao['Válidas Sim (%)'].mean():.2f}%")

print("\n🔹 IBM QUANTUM (hardware real):")
print("-" * 50)
print(f"   • Backend: {df_ibm['Backend'].iloc[0]}")
print("   • Afetado por ruído, decoerência e erros de porta")
print(f"   • Média de soluções válidas: {df_comparacao['Válidas IBM (%)'].mean():.2f}%")

print("\n🔹 IMPACTO DO RUÍDO:")
print("-" * 50)
diff_validas = df_comparacao['Válidas Sim (%)'].mean() - df_comparacao['Válidas IBM (%)'].mean()
print(f"   • Redução em soluções válidas: {diff_validas:.2f}%")
print("   • Hardware real tende a produzir mais soluções inválidas")
print("   • Profundidade do circuito afeta qualidade dos resultados")

print("\n🔹 CONCLUSÕES:")
print("-" * 50)
print("   • QAOA funciona em hardware real, mas com degradação")
print("   • Técnicas de mitigação de erros podem melhorar resultados")
print("   • Para problemas maiores, ruído se torna mais problemático")
print("   • Simulador é útil para desenvolvimento e otimização")

In [ ]:
# =============================================================================
# EXPORTAR RESULTADOS
# =============================================================================

df_comparacao.to_csv('resultados_ibm_quantum.csv', index=False)
print("\n📁 Resultados exportados para 'resultados_ibm_quantum.csv'")

---
## 📝 Notas Importantes

### Diferenças Hardware Real vs Simulador

| Aspecto | Simulador | IBM Quantum |
|---------|-----------|-------------|
| Ruído | Nenhum | Presente |
| Decoerência | Não | Sim |
| Erros de porta | Não | ~0.1-1% por porta |
| Conectividade | Total | Limitada |
| Tempo | Segundos | Minutos (com fila) |

### Limitações do Hardware Atual

- **Profundidade do circuito**: Circuitos muito profundos degradam devido à decoerência
- **Conectividade**: Qubits não estão todos conectados, requer SWAP gates
- **Tempo de coerência**: ~100μs típico, limita operações

### Para Melhorar Resultados

1. Usar backends com menor taxa de erro
2. Aplicar mitigação de erros (ZNE, PEC)
3. Reduzir profundidade do circuito
4. Aumentar número de shots